### Programming HW 2 Problems 4-6 continued from notebook W2_HW2_Prob1-4

In [ ]:
Q4: Index-assisted approximate matching. In practicals, we built a Python class called Index

implementing an ordered-list version of the k-mer index.  The Index class is copied below.

In [ ]:
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [ ]:
We also implemented the pigeonhole principle using Boyer-Moore as our exact matching algorithm.

Implement the pigeonhole principle using \verb|Index|Index to find exact matches for the partitions. Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions). We will use an 8-mer index.

Download the Python module for building a k-mer index. 

https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

Write a function that, given a length-24 pattern P and given an Index object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.

How many times does the string GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

Hint 1: Multiple index hits might direct you to the same match multiple times, but be careful not to count a match more than once.

Hint 2: You can check your work by comparing the output of your new function to that of the naive_2mm function implemented in the previous module.

Q5: Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?
(Don't consider reverse complements.)

Hint: You should be able to use the boyer_moore function (or the slower naive function) to double-check your answer. 

In [2]:
# import bisect module
import bisect

In [3]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'

In [5]:
#Read Genome from FASTA file#
#############################

def readGenome(filename):
    """ This function reads a FASTA file"""
    genome = '' # initialize genome to empty string
    with open(filename,'r') as f:               # Open file as f 
        for line in f:                          # Loop therough and read each line of file f
            if not line[0] == '>':              # If line does not start with ">"
                genome += line.rstrip()         # Add line to the string genome, rstrip removes trailing whitespace from ends of string        
        
    return genome                               # After reading and adding all lines return the string genome


In [9]:
t = readGenome('chr1.GRCh38.excerpt.fasta')

In [12]:
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [13]:
def query_index(p, t, index):
    k = index.k
    offsets =[]
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:
            offsets.append(i)
    return offsets           

In [18]:
index = Index(t, 8)

In [23]:
def approximate_match(p, t, n):   #Takes as arguments pattern, text and the max number of mismatches n
    segment_length = int(round(len(p) /(n+1))) # Set the length of each partition of p, convert to int so that indices are int or it will raise error
    all_matches = set() # create a set to hold all the indices wehere we find a match, wihtout duplicates
    hits = 0
    for i in range(n+1): # For each segment of P, for each iteration move along by the lenght of 1 segment
        #Set bounds of P for the segment we are searching for 
        start = i * segment_length # so if i = 0 and seg_l = 2, start = 0, i = 1, seg_l= 2, start = 1 * 2
        end = min((i+1) * segment_length, len(p)) #to make sure we dont run over end of p, since p might not be a perfect multiple of n+1
        matches = query_index(p[start:end], t, index) # use the boyer moore function and pass in the substring, p_bm obj, t
        hits += len(matches) # sums the numbe of hits for each partition
        # Verification to see that the rest of p matches t with no more than n mismatches
        
        for m in matches:
            #Make sure our location does not let p run off the begining or the end of t
            if m < start or m-start+len(p) > len(t): # if any of this is true then we will run past the beginning or end of t
                continue # if any of the abov is true skip the rest of the loop
                
            mismatches = 0 # To count the mismatches between the rest of p and t
            # Compare part of p before the start(from 0 to start against corresponding position in t)
            for j in range(0, start):
                if not p[j] == t[m-start + j]: # if corresponding positions dont match
                    mismatches += 1 # increment mismatch by 1
                    if mismatches > n: # If the number of mismatches is more than n
                        break  # break out of this inner loop
            # Compare the part of p after the end
            for j in range(end, len(p)): 
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
                        
            if mismatches <= n: # If we have verified on both sides of p and mismatches are less than n
                all_matches.add(m-start) # we add the m - start to get the begining of p for the match to the set all_matches
                
                
    return list(all_matches), hits

In [24]:
all_matches, hits = approximate_match(p, t, 2)

In [27]:
# Problem 4
print(len(all_matches))

19

In [28]:
# Problem 5
print(hits)

90


In [84]:
ind = Index(t, 8)

In [89]:
q1 = ind.query(p[0:])

In [90]:
q2 = ind.query(p[1:])

In [91]:
q3 = ind.query(p[2:])

In [92]:
q = q1+q2+q3

In [94]:
len(q)

41

In [7]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'

In [8]:
t = readGenome('chr1.GRCh38.excerpt.fasta')

Q6: Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings, as we discussed in the "Variations on k-mer indexes" video.  We'll consider subsequences involving every N characters.  For example, if we split \verb|ATATAT|ATATAT into two substring partitions, we would get partitions \verb|ATA|ATA (the first half) and \verb|TAT|TAT (second half).  But if we split \verb|ATATAT|ATATAT into two  subsequences  by taking every other character, we would get \verb|AAA|AAA (first, third and fifth characters) and \verb|TTT|TTT (second, fourth and sixth).

Another way to visualize this is using numbers to show how each character of P is allocated to a partition.  Splitting a length-6 pattern into two substrings could be represented as \verb|111222|111222, and splitting into two subsequences of every other character could be represented as \verb|121212|121212

The following class \verb|SubseqIndex|SubseqIndex is a more general implementation of \verb|Index|Index that additionally handles subsequences. It only considers subsequences that take every Nth character:

In [95]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


For example if we do:

In [ ]:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)

we see:

In [ ]:
[('AAA', 0), ('TTT', 1)]

And if we query this index:

In [ ]:
p = 'TTATAT'
print(ind.query(p[0:]))

we see:

In [ ]:
[]

because the subsequence TAA is not in the index. But if we query with the second subsequence:

In [ ]:
print(ind.query(p[1:]))

we see:

In [ ]:
because the second subsequence TTT is in the index.

Write a function that, given a length-24 pattern P and given a SubseqIndex object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.

When using this function, how many total index hits are there when searching for GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)

Hint: See this notebook for a few examples you can use to test your function.

Answer Prob 6

In [96]:
subseq_ind = SubseqIndex(t, 8, 3)

In [97]:
s_q1 = subseq_ind.query(p[0:])

In [98]:
s_q2 = subseq_ind.query(p[1:])

In [99]:
s_q3 = subseq_ind.query(p[2:])

In [100]:
s_q = s_q1 + s_q2 + s_q3

In [103]:
# P6 
len(s_q)

79

In [139]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences


In [148]:
def approximate_match(p, t, n):   #Takes as arguments pattern, text and the max number of mismatches n
    segment_length = int(round(len(p) /(n+1))) # Set the length of each partition of p, convert to int so that indices are int or it will raise error
    all_matches = set() # create a set to hold all the indices wehere we find a match, wihtout duplicates
    for i in range(n+1): # For each segment of P, for each iteration move along by the lenght of 1 segment
        print(i)
        #Set bounds of P for the segment we are searching for 
        start = i * segment_length # so if i = 0 and seg_l = 2, start = 0, i = 1, seg_l= 2, start = 1 * 2
        print(start)
        end = min((i+1) * segment_length, len(p)) #to make sure we dont run over end of p, since p might not be a perfect multiple of n+1
        print(end)
        matches = naive(p[start:end], t) # use the boyer moore function and pass in the substring, p_bm obj, t
        
    return matches

In [149]:
matches = approximate_match(p, t, 2)

0
0
8
1
8
16
2
16
24


In [147]:
len(matches)

60

In [156]:
m1 = naive(p[0:8], t)

In [152]:
m2 = naive(p[8:16], t)

In [154]:
m3 = naive(p[16:24], t)

In [157]:
m = m1 + m2 + m3

In [158]:
len(m)

90

In [159]:
ma1 = query_index(p[0:8], t, index)

In [160]:
ma2 = query_index(p[8:16], t, index)

In [161]:
ma3 = query_index(p[16:24], t, index)

In [162]:
ma = ma1 + ma2 + ma3

In [163]:
len(ma)

90

In [169]:
# Problem 5
# Function to get all the matches 
n = 2
matches = []
for i in range (n+1):
    segment_length =int(round(len(p) / (n+1)))
    start = i * segment_length
    end = min((i+1) * segment_length, len(p))
    match = query_index(p[start:end], t, index)
    matches.append(match)  

In [170]:
len(matches)

3

Matches has three nested lists and we need to sum the length of each to get the numnber of total hits

In [176]:
match1 = matches[0]

In [177]:
match2 = matches[1]

In [178]:
match3 = matches[2]

In [179]:
matches = match1 + match2 + match3

In [180]:
len(matches)

90

In [46]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


In [47]:
def query_subseq(p, t, index):
    k = index.k
    offsets =[]
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:
            offsets.append(i)
    return offsets           

In [48]:
subseq_index = SubseqIndex(t, 8, 3)

In [49]:
def approximate_match(p, t, n):   #Takes as arguments pattern, text and the max number of mismatches n
    segment_length = int(round(len(p) /(n+1))) # Set the length of each partition of p, convert to int so that indices are int or it will raise error
    all_matches = set() # create a set to hold all the indices wehere we find a match, wihtout duplicates
    hits = 0
    for i in range(n+1): # For each segment of P, for each iteration move along by the lenght of 1 segment
        #Set bounds of P for the segment we are searching for 
        start = i * segment_length # so if i = 0 and seg_l = 2, start = 0, i = 1, seg_l= 2, start = 1 * 2
        end = min((i+1) * segment_length, len(p)) #to make sure we dont run over end of p, since p might not be a perfect multiple of n+1
        matches = query_subseq(p[start:end], t, subseq_index) # use the boyer moore function and pass in the substring, p_bm obj, t
        hits += len(matches) # sums the numbe of hits for each partition
        # Verification to see that the rest of p matches t with no more than n mismatches
        
        for m in matches:
            #Make sure our location does not let p run off the begining or the end of t
            if m < start or m-start+len(p) > len(t): # if any of this is true then we will run past the beginning or end of t
                continue # if any of the abov is true skip the rest of the loop
                
            mismatches = 0 # To count the mismatches between the rest of p and t
            # Compare part of p before the start(from 0 to start against corresponding position in t)
            for j in range(0, start):
                if not p[j] == t[m-start + j]: # if corresponding positions dont match
                    mismatches += 1 # increment mismatch by 1
                    if mismatches > n: # If the number of mismatches is more than n
                        break  # break out of this inner loop
            # Compare the part of p after the end
            for j in range(end, len(p)): 
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
                        
            if mismatches <= n: # If we have verified on both sides of p and mismatches are less than n
                all_matches.add(m-start) # we add the m - start to get the begining of p for the match to the set all_matches
                
                
    return list(all_matches), hits

In [50]:
all_matches, hits = approximate_match(p, t, 2)

In [51]:
# Problem 6 Gives the number of total index hits
print(hits)

0


In [52]:
# Total number of exact matches with upto 2 mismatches
print(all_matches) 

[]
